<a href="https://colab.research.google.com/github/hpnightowl/Owlsbooks/blob/master/weblog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the PyDrive wrapper & import libraries.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#1bM4P-UhYbwpb8avaUH9RlkGwQuPlQ_o3 - jul
#1lCrS678Y7n93iFXO568fN78zcY2ffbPY - aug
file_id = '1lCrS678Y7n93iFXO568fN78zcY2ffbPY'
downloaded = drive.CreateFile({'id':file_id})
downloaded.FetchMetadata(fetch_all=True)
downloaded.GetContentFile(downloaded.metadata['title'])

In [ ]:
import os
import re
import pandas as pd
import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
FileList = glob.glob("access_log_*95")
print(FileList)

['access_log_Aug95', 'access_log_Jul95']


In [ ]:
data_gen = (pd.read_csv(FileName, sep="!", encoding="ISO-8859-1", header=None) for FileName in FileList)
data = pd.concat(data_gen, ignore_index = True)
data.columns = ["Data"]

In [ ]:
pd.set_option("display.max_colwidth", None)
data.head(5)

,Data
0,"in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] ""GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0"" 200 1839"
1,"uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] ""GET / HTTP/1.0"" 304 0"
2,"uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] ""GET /images/ksclogo-medium.gif HTTP/1.0"" 304 0"
3,"uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] ""GET /images/MOSAIC-logosmall.gif HTTP/1.0"" 304 0"
4,"uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] ""GET /images/USA-logosmall.gif HTTP/1.0"" 304 0"


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461613 entries, 0 to 3461612
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Data    object
dtypes: object(1)
memory usage: 26.4+ MB


In [ ]:
#Extracting Bytes from the given logfile
bytesRegex = r"\s(\d+)$"
ListofBytes = [re.search(bytesRegex, record).group(1)
             if re.search(bytesRegex, record)
             else "No match found"
             for record in data["Data"]]

#Extracting all StatusCodes from the given logfile
statusRegex = r"\s(\d{3})\s"

statuscodes = [re.search(statusRegex, record).group(1)
            if re.search(statusRegex, record)
            else "No match found"
            for record in data["Data"]]

#Extracting all dates from given logfile
dateRegex = r"(\d{2}[\/]\w*[\/]\d{4}:\d{2}:\d{2}:\d{2})"

dates = [re.search(dateRegex, record).group(1)
             if re.search(dateRegex, record)
             else "No match found"
             for record in data["Data"]]

#Extracting all hostnames from given logfile
HostRegex = r"(^\S+)\s"
Hostnames = [re.search(HostRegex, record).group(1)
               if re.search(HostRegex, record)
               else "No match found"
               for record in data["Data"]]

In [ ]:
data["Host"] = data["Data"].str.extract(HostRegex, expand = True)
data["Date"] = data["Data"].str.extract(dateRegex, expand = True)
data["Status"] = data["Data"].str.extract(statusRegex, expand = True)
data["Bytes"] = data["Data"].str.extract(bytesRegex, expand = True)

In [ ]:
real_Data = data.filter(['Host','Date','Status','Bytes'], axis=1)

In [ ]:
real_Data

,Host,Date,Status,Bytes
0,in24.inetnebr.com,01/Aug/1995:00:00:01,200,1839
1,uplherc.upl.com,01/Aug/1995:00:00:07,304,0
2,uplherc.upl.com,01/Aug/1995:00:00:08,304,0
3,uplherc.upl.com,01/Aug/1995:00:00:08,304,0
4,uplherc.upl.com,01/Aug/1995:00:00:08,304,0
...,...,...,...,...
3461608,163.205.53.14,28/Jul/1995:13:32:23,200,1204
3461609,tiger2.ocs.lsu.edu,28/Jul/1995:13:32:23,200,8677
3461610,199.0.2.27,28/Jul/1995:13:32:23,200,5866
3461611,tornado.umd.edu,28/Jul/1995:13:32:25,200,5494
